# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sklearn

# Import Dataset

In [2]:
# read dataset from source
df = pd.read_csv('/kaggle/input/melbourne-housing-snapshot/melb_data.csv')

In [3]:
# output a sample view of df
df.sample(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
12279,Doncaster,13 Winbrook Ct,5,h,1820000.0,S,Buxton,3/09/2017,12.4,3108.0,...,3.0,2.0,630.0,NaN,NaN,NaN,-37.78151,145.12515,Eastern Metropolitan,9028.0
11239,Vermont,11 Manhattan Sq,4,h,980000.0,S,hockingstuart,12/08/2017,17.2,3133.0,...,2.0,0.0,539.0,192.0,1978.0,Whitehorse,-37.83459,145.21264,Eastern Metropolitan,4181.0
6393,Toorak,6 Leighton Ct,3,h,4200000.0,S,Melbourne,23/04/2016,4.6,3142.0,...,4.0,2.0,565.0,331.0,2005.0,Stonnington,-37.84480,145.01230,Southern Metropolitan,7217.0
11038,Glen Iris,20 Madeline St,3,h,1690000.0,S,Jellis,12/08/2017,7.3,3146.0,...,1.0,2.0,625.0,NaN,NaN,Boroondara,-37.85728,145.08743,Southern Metropolitan,10412.0
4908,Preston,10 Tyler St,3,h,550000.0,VB,Love,8/10/2016,8.8,3072.0,...,2.0,1.0,164.0,NaN,NaN,Darebin,-37.73350,145.03740,Northern Metropolitan,14577.0


# Data Clean and Prepaire

In [4]:
# check data type
df.dtypes

Suburb            object
Address           object
Rooms              int64
Type              object
Price            float64
Method            object
SellerG           object
Date              object
Distance         float64
Postcode         float64
Bedroom2         float64
Bathroom         float64
Car              float64
Landsize         float64
BuildingArea     float64
YearBuilt        float64
CouncilArea       object
Lattitude        float64
Longtitude       float64
Regionname        object
Propertycount    float64
dtype: object

In [5]:
# change data type
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

In [6]:
# check data type of 'Date' column
df['Date'].dtypes

dtype('<M8[ns]')

In [7]:
# check duplicated values
df.duplicated().any(), df.duplicated().sum()

(False, 0)

In [8]:
# check data validity
df.select_dtypes(include='object').nunique()

Suburb           314
Address        13378
Type               3
Method             5
SellerG          268
CouncilArea       33
Regionname         8
dtype: int64

In [9]:
# print the unique values for columns that have unique values less than 15 count
for x in np.array(df.select_dtypes(include='object').columns):
    # check the condition
    if df.select_dtypes(include='object')[x].nunique() < 15:
        # print column name
        print(x)
        # print unique values
        print(df.select_dtypes(include='object')[x].unique())


Type
['h' 'u' 't']
Method
['S' 'SP' 'PI' 'VB' 'SA']
Regionname
['Northern Metropolitan' 'Western Metropolitan' 'Southern Metropolitan'
 'Eastern Metropolitan' 'South-Eastern Metropolitan' 'Eastern Victoria'
 'Northern Victoria' 'Western Victoria']


In [10]:
# get information about numerical columns
df.select_dtypes(exclude='object').describe().round(2)

,Rooms,Price,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.00,13580.00,13580,13580.00,13580.00,13580.00,13580.00,13518.00,13580.00,7130.00,8205.00,13580.00,13580.00,13580.00
mean,2.94,1075684.08,2017-01-28 20:25:03.622974976,10.14,3105.30,2.91,1.53,1.61,558.42,151.97,1964.68,-37.81,145.00,7454.42
min,1.00,85000.00,2016-01-28 00:00:00,0.00,3000.00,0.00,0.00,0.00,0.00,0.00,1196.00,-38.18,144.43,249.00
25%,2.00,650000.00,2016-09-10 00:00:00,6.10,3044.00,2.00,1.00,1.00,177.00,93.00,1940.00,-37.86,144.93,4380.00
50%,3.00,903000.00,2017-03-04 00:00:00,9.20,3084.00,3.00,1.00,2.00,440.00,126.00,1970.00,-37.80,145.00,6555.00
75%,3.00,1330000.00,2017-06-24 00:00:00,13.00,3148.00,3.00,2.00,2.00,651.00,174.00,1999.00,-37.76,145.06,10331.00
max,10.00,9000000.00,2017-09-23 00:00:00,48.10,3977.00,20.00,8.00,10.00,433014.00,44515.00,2018.00,-37.41,145.53,21650.00
std,0.96,639310.72,NaN,5.87,90.68,0.97,0.69,0.96,3990.67,541.01,37.27,0.08,0.10,4378.58


In [11]:
# plot box plot for numerical columns
fig = make_subplots(rows=len(df.select_dtypes(exclude='object').columns), cols=1, shared_xaxes=False)

i = 1
for x in np.array(df.select_dtypes(exclude='object').columns):
    fig.add_trace(go.Box(x=df[x], name=x), row=i, col=1)
    i+=1

fig.update_layout(height=3000)
fig.show()

# Data Analysis

In [12]:
df.sample()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
3585,Kew,3/81 Derby St,2,u,630000.0,PI,Marshall,2017-03-04,5.6,3101.0,...,1.0,2.0,91.0,NaN,1970.0,Boroondara,-37.8003,145.0375,Southern Metropolitan,10331.0


In [13]:
px.imshow(df.select_dtypes(exclude='object').corr(numeric_only=True), color_continuous_scale='Blues', text_auto=True, aspect=True)

In [14]:
px.scatter(data_frame=df, x='Price', y='BuildingArea', size='Price', color='Type', facet_col='Type', trendline='ols')

# Building Model

In [15]:
# select object columns and save it numpy array (object_col)
object_col = np.array(df.select_dtypes(include='object').columns)
# print (object_col)
print(object_col)

['Suburb' 'Address' 'Type' 'Method' 'SellerG' 'CouncilArea' 'Regionname']


In [16]:
# print unique values that less than 15
for x in object_col:
    # print column name
    print('column name: ' + x)
    # check num of unique values in the column
    # if the num of unique values are less than 15
    if df[x].nunique() < 15:
        # print num of unique values
        print('num of unique values: ' + str(df[x].nunique()))
        # print the unique values
        print(df[x].unique())
        # if the num of unique values are more than 15
    else:
        print('unique values are more than 15')

column name: Suburb
unique values are more than 15
column name: Address
unique values are more than 15
column name: Type
num of unique values: 3
['h' 'u' 't']
column name: Method
num of unique values: 5
['S' 'SP' 'PI' 'VB' 'SA']
column name: SellerG
unique values are more than 15
column name: CouncilArea
unique values are more than 15
column name: Regionname
num of unique values: 8
['Northern Metropolitan' 'Western Metropolitan' 'Southern Metropolitan'
 'Eastern Metropolitan' 'South-Eastern Metropolitan' 'Eastern Victoria'
 'Northern Victoria' 'Western Victoria']


In [17]:
# drop columns that have unique values more than 15
for x in df.columns:
    # check if column as x dtypes is object and has unique values count more than 15
    if (df[x].dtypes == 'object') & (df[x].nunique() > 15):
        # drop column named as x
        df.drop(columns=x, inplace=True)

In [18]:
# select object columns and save it numpy array (object_col)
object_col = df.select_dtypes(include='object').columns
# print (object_col)
print(object_col)

Index(['Type', 'Method', 'Regionname'], dtype='object')


In [19]:
# encoding for object columns
for col in df.columns:
    if col in object_col:
        # encoding
        encoding_col = pd.get_dummies(df[col], prefix='encod_' + col, prefix_sep='_', drop_first=True)
        # concat new column with df
        df = pd.concat([df, encoding_col], axis=1)
        # drop origenal object column
        df.drop(columns=col, inplace=True)

In [20]:
# check null values
df.isnull().any(), df.isnull().sum()

(Rooms                                          False
 Price                                          False
 Date                                           False
 Distance                                       False
 Postcode                                       False
 Bedroom2                                       False
 Bathroom                                       False
 Car                                             True
 Landsize                                       False
 BuildingArea                                    True
 YearBuilt                                       True
 Lattitude                                      False
 Longtitude                                     False
 Propertycount                                  False
 encod_Type_t                                   False
 encod_Type_u                                   False
 encod_Method_S                                 False
 encod_Method_SA                                False
 encod_Method_SP            

In [21]:
# import (SimpleImputer) subpackege to impute miss data
from sklearn.impute import SimpleImputer

In [22]:
# creat object of (SimpleImputer)
my_SimpleImputer = SimpleImputer()

In [23]:
# import (train_test_split) subpakege to prepaire test and train dataset
from sklearn.model_selection import train_test_split

In [24]:
# save the column that want to predict it variable named y
y = df['Price']

In [25]:
# # save the rest of columns in variable named X
X = df.drop(columns=['Price', 'Date'])

In [26]:
# prepaire X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [27]:
# check shape
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((9098, 25), (4482, 25), (9098,), (4482,))

In [28]:
# use (my_SimpleImputer) to impute miss data in X_train & X_test
X_train_SimpleImputer = pd.DataFrame(my_SimpleImputer.fit_transform(X_train))
X_test_SimpleImputer = pd.DataFrame(my_SimpleImputer.transform(X_test))

In [29]:
# set columns name for (X_train_SimpleImputer) & (X_test_SimpleImputer) as same as X_train & X_test
X_train_SimpleImputer.columns = X_train.columns
X_test_SimpleImputer.columns = X_test.columns

In [30]:
# import Algorithm
from sklearn.ensemble import RandomForestRegressor

In [31]:
# import metrics to evaluate model
from sklearn.metrics import mean_absolute_error

In [32]:
# build, fit and predict
for n in [1, 10, 100, 1000]:
    # building model
    model = RandomForestRegressor(n_estimators=n, random_state=1)
    # fit model
    model.fit(X_train_SimpleImputer, y_train)
    # predict
    predicted = model.predict(X_test_SimpleImputer)
    # evaluate model
    mae = mean_absolute_error(y_true=y_test, y_pred=predicted)
    # print output for every n
    print('n_estimators = %d \t\t mae: %d'%(n, mae))

n_estimators = 1 		 mae: 240405
n_estimators = 10 		 mae: 176435
n_estimators = 100 		 mae: 167326
n_estimators = 1000 		 mae: 166639


model with n_estimators = 1000 is the best one